
<h2> Capstone Project - The Battle of Two Neighborhoods</h2>

<hr /> 

<b>Aim:</b>
A local business is seeking crucial information in their decision making concerning expanding further within the Atlanta Metro area. Specifically, the business owner is requesting detail information comparing the two cities of Smyrna and Cedartown.


<b>Problem Statement:</b>
With the recent closure of all of the Performance Bicycle retail stores (largest national retailer of bicycles and accessories), the Atlanta metro has lost 4 major bicycles shops. Today, many people within their metro communities are without any close-by bicycle shops due to the closures.

A local bicycle shop in the south metro community of Peachtree City, Hometown Bikes, is investigating into expand into the Atlanta Metro area. They think that the time is right to expand their business further into the metro area. In addition to expanding, they are aiming to be located near the Silver Comet trail in order to provide support and services to travelers along the trail. In addition to providing bicycle service and parts, the business is considering providing other such services as coffee, food, guided rides, and possible overnight accommodations in the future (depending upon the data results). They are requesting location information of competitors’ venues within the two cities in order to determine which area offers the most opportunity for their business.

The Silver Comet and Chief Ladiga trails join to form one continuous 94-mile (151 km) trail from Smyrna, Georgia (Atlanta area) to Anniston, Alabama, which together form the second longest paved rail trail in the U.S.  Hometown Bikes is only considering a retail location within the state of Georgia, therefore they have picked Cedartown as the western most potential location.

<b>Objectives:</b> I will begin by obtaining the geojson data for both Smyna and Cedartown. 2) Then, analyzing the data using the Foursquare API. 3)  Afterwards I will use clustering to identify each area and its category. The data source that I will be using is from Foursquare, which is a technology company that uses location intelligence to build meaningful consumer experiences and business solutions. 

Using techniques such as K-means clustering, I will able to get results about competitors’ venues within the two areas that would be attractive to the target customers. k-means clustering aims to partition n observations into k clusters in which each observation belongs to the cluster with the nearest mean, serving as a prototype of the cluster. 

<b>Conclusions:</b>
By making use of Foursquare data sets and data science methodology, I have successfully created information to assist the business owner in selecting a new retail location that will serve the needs of travelers along the Silver Comet Trail. 

<br />
<hr />
<br />

In [148]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
!pip install folium
import requests
#from bs4 import BeautifulSoup
import lxml
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import json
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframev
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
print("All libraries installed and imported ! ")

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

All libraries installed and imported ! 


In [149]:

CLIENT_ID = 'NSA1IUUONWX3I4AIFRKU1404QRRDXRBD41SIM423PX0LOY2W' # your Foursquare ID
CLIENT_SECRET = 'KJYXQ1UKUVANIWMI3OU1OIJYLKNKIJAZYHCWMM3YAAIB4OHK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 30
radius=8047

<h2>The business owners is specifically requesting the following Foursquare Categories.</h2>

Bike shops:
4bf58dd8d48988d115951735

Sporting Goods Shop:
4bf58dd8d48988d1f2941735

Coffee Shops:
4bf58dd8d48988d1e0931735

Campgrounds:
4bf58dd8d48988d1e4941735

Hotels:
4bf58dd8d48988d1fa931735

Bus Station:
4bf58dd8d48988d1fe931735


These can be found on the Foursquare Developer resources page:
https://developer.foursquare.com/docs/resources/categories

In [150]:
#assign categories to list -> dataframe
categories_list =[{'category_id':'4bf58dd8d48988d115951735' ,'desc':'Bike shop', 'rank':5 },
                  {'category_id':'4bf58dd8d48988d1f2941735' ,'desc':'Sporting Goods', 'rank':5 },
                  {'category_id':'4bf58dd8d48988d1e0931735' ,'desc':'Coffee Shop', 'rank':4 },
                  {'category_id':'4bf58dd8d48988d1e4941735' ,'desc':'Campground', 'rank':3 },
                  {'category_id':'4bf58dd8d48988d1fa931735' ,'desc':'Hotel', 'rank':3 },
                    {'category_id':'4bf58dd8d48988d1fe931735' ,'desc':'Bus Station', 'rank':3 }]


categories_df = pd.DataFrame(categories_list)

categories = categories_df['category_id'].str.cat(sep=',')

categories


'4bf58dd8d48988d115951735,4bf58dd8d48988d1f2941735,4bf58dd8d48988d1e0931735,4bf58dd8d48988d1e4941735,4bf58dd8d48988d1fa931735,4bf58dd8d48988d1fe931735'

In [195]:
#Due to location and population, Smyrna will be grade by a different scale.
def smyrna_getGrade(s):
    if s['likes count']<=25:
        return 'poor'
    if s['likes count']<=50:
        return 'average'
    if s['likes count']<=100:
        return 'good' 
    if s['likes count']>100:
        return 'awesome'

    
def cedartown_getGrade(s):
    if s['likes count']<=2:
        return 'poor'
    if s['likes count']<=10:
        return 'average'
    if s['likes count']<=15:
        return 'good' 
    if s['likes count']>15:
        return 'awesome'
    
    

<p>
<h3>Smyrna Neighborhood Data</h3> </p>
<hr />

In [152]:
#Smyrna coordinates
lat=33.8413889
long=-84.51666666666667
radius=8047 #5 miles due to the density of the area
smyrna_url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, long, VERSION, categories, radius, LIMIT)
smyrna_url 


'https://api.foursquare.com/v2/venues/search?client_id=NSA1IUUONWX3I4AIFRKU1404QRRDXRBD41SIM423PX0LOY2W&client_secret=KJYXQ1UKUVANIWMI3OU1OIJYLKNKIJAZYHCWMM3YAAIB4OHK&ll=33.8413889,-84.51666666666667&v=20180605&categoryId=4bf58dd8d48988d115951735,4bf58dd8d48988d1f2941735,4bf58dd8d48988d1e0931735,4bf58dd8d48988d1e4941735,4bf58dd8d48988d1fa931735,4bf58dd8d48988d1fe931735&radius=8047&limit=30'

In [153]:
# Smyrna location results
smyrna_results = requests.get(smyrna_url).json()
smyrna_results

{'meta': {'code': 200, 'requestId': '5d5f29476bdee6002c78accd'},
 'response': {'venues': [{'id': '4b915b9af964a520f1b533e3',
    'name': "Dunkin'",
    'location': {'address': '3300 S Cobb Dr SE',
     'crossStreet': 'Concord',
     'lat': 33.87338725112799,
     'lng': -84.53095724785044,
     'labeledLatLngs': [{'label': 'display',
       'lat': 33.87338725112799,
       'lng': -84.53095724785044}],
     'distance': 3799,
     'postalCode': '30080',
     'cc': 'US',
     'city': 'Smyrna',
     'state': 'GA',
     'country': 'United States',
     'formattedAddress': ['3300 S Cobb Dr SE (Concord)',
      'Smyrna, GA 30080',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d148941735',
      'name': 'Donut Shop',
      'pluralName': 'Donut Shops',
      'shortName': 'Donuts',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/donuts_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1566517575',
    'hasPerk': False},
   {'id':

In [154]:
smyrna_venues = smyrna_results['response']['venues']

# tranform venues into a dataframe
smyrna_df = json_normalize(smyrna_venues)
smyrna_df

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d148941735', 'name': 'D...",False,4b915b9af964a520f1b533e3,3300 S Cobb Dr SE,US,Smyrna,United States,Concord,3799,"[3300 S Cobb Dr SE (Concord), Smyrna, GA 30080...","[{'label': 'display', 'lat': 33.87338725112799...",33.873387,-84.530957,30080,GA,Dunkin',v-1566517575,NaN
1,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,4a873ed3f964a520700320e3,1680 Spring Rd SE Ste B,US,Smyrna,United States,at Jonquil Dr,4689,"[1680 Spring Rd SE Ste B (at Jonquil Dr), Smyr...","[{'label': 'display', 'lat': 33.88222257589188...",33.882223,-84.504198,30080,GA,Rev Coffee,v-1566517575,NaN
2,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,4aeb28ddf964a52025bf21e3,4515 West Village Way,US,Smyrna,United States,NaN,2496,"[4515 West Village Way, Smyrna, GA 30080, Unit...","[{'label': 'display', 'lat': 33.84422988325082...",33.844230,-84.489886,30080,GA,Starbucks,v-1566517575,NaN
3,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4ae2ee59f964a520b68f21e3,2450 Galleria Pkwy SE,US,Atlanta,United States,Cobb Pkwy. SE,6768,"[2450 Galleria Pkwy SE (Cobb Pkwy. SE), Atlant...","[{'label': 'display', 'lat': 33.8845775, 'lng'...",33.884577,-84.465127,30339,GA,Renaissance Atlanta Waverly Hotel & Convention...,v-1566517575,NaN
4,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,4a787a35f964a520bce51fe3,2680 Cobb Pkwy SE,US,Smyrna,United States,NaN,6480,"[2680 Cobb Pkwy SE, Smyrna, GA 30080, United S...","[{'label': 'display', 'lat': 33.8875367260119,...",33.887537,-84.473934,30080,GA,Starbucks,v-1566517575,NaN
5,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4a9b172df964a5205b3420e3,2844 Cobb Pkwy SE,US,Atlanta,United States,NaN,6401,"[2844 Cobb Pkwy SE, Atlanta, GA 30339, United ...","[{'label': 'display', 'lat': 33.88294840385119...",33.882948,-84.468797,30339,GA,Sheraton Suites Galleria-Atlanta,v-1566517575,NaN
6,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,4a69b677f964a520fecb1fe3,2495 Cumberland Pkwy SE,US,Atlanta,United States,NaN,4469,"[2495 Cumberland Pkwy SE, Atlanta, GA 30339, U...","[{'label': 'display', 'lat': 33.86488251, 'lng...",33.864883,-84.477467,30339,GA,Starbucks,v-1566517575,NaN
7,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4b15e5e7f964a5206cb523e3,2045 S Park Place,US,Atlanta,United States,NaN,7829,"[2045 S Park Place, Atlanta, GA 30339, United ...","[{'label': 'display', 'lat': 33.90341165492204...",33.903412,-84.476728,30339,GA,Courtyard Atlanta Marietta/Windy Hill,v-1566517575,182956630
8,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4b77788af964a520889c2ee3,200 Interstate North Pkwy SE,US,Atlanta,United States,at Windy Hill Rd,7843,[200 Interstate North Pkwy SE (at Windy Hill R...,"[{'label': 'display', 'lat': 33.89851169777089...",33.898512,-84.466989,30339,GA,Atlanta Marriott Northwest,v-1566517575,89159026
9,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,55d2b694498e2650ac9975c1,2857 Paces Ferry Rd SE,US,Atlanta,United States,NaN,5031,"[2857 Paces Ferry Rd SE, Atlanta, GA 30339, Un...","[{'label': 'display', 'lat': 33.86632250184305...",33.866323,-84.471271,30339,GA,Hotel Indigo Atlanta - Vinings,v-1566517575,NaN


<b>Preparing the Smyrna dataset </b>

In [155]:
#Create function to be used in preparing the data
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

                 
 

filtered_columns1 = ['id','name', 'categories', 'location.lat', 'location.lng']
smyrna_df_filtered = smyrna_df.loc[:, filtered_columns1]


# filter the category for each row
smyrna_df_filtered['categories'] = smyrna_df_filtered.apply(get_category_type, axis=1)


# clean column names by keeping only last term
smyrna_df_filtered.columns = [column.split('.')[-1] for column in smyrna_df_filtered.columns]

smyrna_df_filtered.head(15)



,id,name,categories,lat,lng
0,4b915b9af964a520f1b533e3,Dunkin',Donut Shop,33.873387,-84.530957
1,4a873ed3f964a520700320e3,Rev Coffee,Coffee Shop,33.882223,-84.504198
2,4aeb28ddf964a52025bf21e3,Starbucks,Coffee Shop,33.844230,-84.489886
3,4ae2ee59f964a520b68f21e3,Renaissance Atlanta Waverly Hotel & Convention...,Hotel,33.884577,-84.465127
4,4a787a35f964a520bce51fe3,Starbucks,Coffee Shop,33.887537,-84.473934
5,4a9b172df964a5205b3420e3,Sheraton Suites Galleria-Atlanta,Hotel,33.882948,-84.468797
6,4a69b677f964a520fecb1fe3,Starbucks,Coffee Shop,33.864883,-84.477467
7,4b15e5e7f964a5206cb523e3,Courtyard Atlanta Marietta/Windy Hill,Hotel,33.903412,-84.476728
8,4b77788af964a520889c2ee3,Atlanta Marriott Northwest,Hotel,33.898512,-84.466989
9,55d2b694498e2650ac9975c1,Hotel Indigo Atlanta - Vinings,Hotel,33.866323,-84.471271


In [156]:
smyrna_venue_id_list = smyrna_df_filtered['id'].tolist()
smyrna_venue_id_list

['4b915b9af964a520f1b533e3',
 '4a873ed3f964a520700320e3',
 '4aeb28ddf964a52025bf21e3',
 '4ae2ee59f964a520b68f21e3',
 '4a787a35f964a520bce51fe3',
 '4a9b172df964a5205b3420e3',
 '4a69b677f964a520fecb1fe3',
 '4b15e5e7f964a5206cb523e3',
 '4b77788af964a520889c2ee3',
 '55d2b694498e2650ac9975c1',
 '52cc49b0498effbb1bc86537',
 '4a8d6d36f964a5209b0f20e3',
 '4b82031ff964a5209ec730e3',
 '4b13b93df964a520b19823e3',
 '518b7e5c498ee42b3058711f',
 '4ba359b3f964a520a93638e3',
 '4bd87b1c5cf276b0845d9d00',
 '4a071978f964a52025731fe3',
 '4bc33da2dce4eee12e82719d',
 '4fc94582d4f24895b4467f21',
 '4bcf7ebb0ffdce725a8cb2c0',
 '51070d9ee4b05470fd8d5f19',
 '4bd5c63f9649ce72dba7511d',
 '5a4dd1e50802d413e0f8e174',
 '4bb79b9498c7ef3ba10d3002',
 '4cb62a9086e09eb044a562e2',
 '4bc33d9cdce4eee12d82719d',
 '4ec7c2ad0e017e2f9a349738',
 '5829ba187d7fb761d009fbfe',
 '4a7c4cacf964a5202eec1fe3']

In [157]:
url_list = []
likes_count_list = []

for i in smyrna_venue_id_list:
    venue_url = 'https://api.foursquare.com/v2/venues/{}/likes?client_id={}&client_secret={}&v={}'.format(i, CLIENT_ID, CLIENT_SECRET, VERSION)
    url_list.append(venue_url)
for link in url_list:
    result = requests.get(link).json()
    likes = result['response']['likes']['count']
    likes_count_list.append(likes)
print(likes_count_list)

[42, 256, 146, 182, 147, 61, 204, 23, 76, 22, 34, 110, 32, 49, 83, 13, 21, 112, 11, 20, 43, 42, 28, 12, 84, 32, 13, 0, 5, 90]


In [158]:
print(len(likes_count_list))
print(len(smyrna_venue_id_list))

30
30


In [159]:
# add in the likes count
smyrna_df_filtered['likes count'] = likes_count_list
smyrna_df_filtered.head()

,id,name,categories,lat,lng,likes count
0,4b915b9af964a520f1b533e3,Dunkin',Donut Shop,33.873387,-84.530957,42
1,4a873ed3f964a520700320e3,Rev Coffee,Coffee Shop,33.882223,-84.504198,256
2,4aeb28ddf964a52025bf21e3,Starbucks,Coffee Shop,33.844230,-84.489886,146
3,4ae2ee59f964a520b68f21e3,Renaissance Atlanta Waverly Hotel & Convention...,Hotel,33.884577,-84.465127,182
4,4a787a35f964a520bce51fe3,Starbucks,Coffee Shop,33.887537,-84.473934,147


In [177]:
smyrna_df_filtered['grade']=smyrna_df_filtered.apply(smyrna_getGrade, axis=1)

In [178]:
smyrna_df_filtered.head()

,id,name,categories,lat,lng,likes count,grade,cluster
0,4b915b9af964a520f1b533e3,Dunkin',Donut Shop,33.873387,-84.530957,42,average,2
1,4a873ed3f964a520700320e3,Rev Coffee,Coffee Shop,33.882223,-84.504198,256,awesome,1
2,4aeb28ddf964a52025bf21e3,Starbucks,Coffee Shop,33.844230,-84.489886,146,awesome,1
3,4ae2ee59f964a520b68f21e3,Renaissance Atlanta Waverly Hotel & Convention...,Hotel,33.884577,-84.465127,182,awesome,1
4,4a787a35f964a520bce51fe3,Starbucks,Coffee Shop,33.887537,-84.473934,147,awesome,1


In [204]:

# one hot encoding
smyrna_to_onehot = pd.get_dummies(smyrna_df_filtered[['grade']], prefix="", prefix_sep="")

# add vaenue name column back to dataframe
smyrna_to_onehot['Name'] = smyrna_df_filtered['name'] 

# move venue name column to the first column
fixed_columns = [smyrna_to_onehot.columns[-1]] + list(smyrna_to_onehot.columns[:-1])
smyrna_to_onehot = smyrna_to_onehot[fixed_columns]

smyrna_to_onehot.head()

,Name,average,awesome,good,poor
0,Dunkin',1,0,0,0
1,Rev Coffee,0,1,0,0
2,Starbucks,0,1,0,0
3,Renaissance Atlanta Waverly Hotel & Convention...,0,1,0,0
4,Starbucks,0,1,0,0


In [205]:
smyrna_cluster_df = smyrna_to_onehot.drop('Name', axis=1)
k_clusters = 3

# run k-means clustering
kmeans = KMeans(n_clusters=k_clusters, random_state=0).fit(smyrna_cluster_df)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]


array([2, 0, 0, 0, 0, 0, 0, 1, 0, 1], dtype=int32)

In [206]:
smyrna_df_filtered['cluster'] = kmeans.labels_
smyrna_df_filtered.head(8)

,id,name,categories,lat,lng,likes count,grade,cluster
0,4b915b9af964a520f1b533e3,Dunkin',Donut Shop,33.873387,-84.530957,42,average,2
1,4a873ed3f964a520700320e3,Rev Coffee,Coffee Shop,33.882223,-84.504198,256,awesome,0
2,4aeb28ddf964a52025bf21e3,Starbucks,Coffee Shop,33.844230,-84.489886,146,awesome,0
3,4ae2ee59f964a520b68f21e3,Renaissance Atlanta Waverly Hotel & Convention...,Hotel,33.884577,-84.465127,182,awesome,0
4,4a787a35f964a520bce51fe3,Starbucks,Coffee Shop,33.887537,-84.473934,147,awesome,0
5,4a9b172df964a5205b3420e3,Sheraton Suites Galleria-Atlanta,Hotel,33.882948,-84.468797,61,good,0
6,4a69b677f964a520fecb1fe3,Starbucks,Coffee Shop,33.864883,-84.477467,204,awesome,0
7,4b15e5e7f964a5206cb523e3,Courtyard Atlanta Marietta/Windy Hill,Hotel,33.903412,-84.476728,23,poor,1


In [207]:
Smyrna_nearby_venues_sort_by_likes = smyrna_df_filtered.sort_values('likes count', ascending=False)

In [208]:
Smyrna_nearby_venues_sort_by_likes

,id,name,categories,lat,lng,likes count,grade,cluster
1,4a873ed3f964a520700320e3,Rev Coffee,Coffee Shop,33.882223,-84.504198,256,awesome,0
6,4a69b677f964a520fecb1fe3,Starbucks,Coffee Shop,33.864883,-84.477467,204,awesome,0
3,4ae2ee59f964a520b68f21e3,Renaissance Atlanta Waverly Hotel & Convention...,Hotel,33.884577,-84.465127,182,awesome,0
4,4a787a35f964a520bce51fe3,Starbucks,Coffee Shop,33.887537,-84.473934,147,awesome,0
2,4aeb28ddf964a52025bf21e3,Starbucks,Coffee Shop,33.844230,-84.489886,146,awesome,0
17,4a071978f964a52025731fe3,Starbucks,Coffee Shop,33.900700,-84.477840,112,awesome,0
11,4a8d6d36f964a5209b0f20e3,Starbucks,Coffee Shop,33.849339,-84.428559,110,awesome,0
29,4a7c4cacf964a5202eec1fe3,Goldberg's Bagels & Deli,Bagel Shop,33.848083,-84.428900,90,good,0
24,4bb79b9498c7ef3ba10d3002,Starbucks,Coffee Shop,33.856094,-84.582118,84,good,0
14,518b7e5c498ee42b3058711f,Starbucks,Coffee Shop,33.916546,-84.466840,83,good,0


<p>
<h3> Cedartown Neighborhood Data </h3> </p>
<hr />

In [184]:
#Cedartown coordinates
lat=33.9521417
long=-85.38857777777778
#radius=24140 #15 miles
radius=32187 #20 miles #4bf58dd8d48988d1e0931735,
#Existing Bike shops
cedartown_url1 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, long, VERSION, categories, radius, LIMIT)
cedartown_url1 
cedartown_results = requests.get(cedartown_url1).json()
cedartown_results


{'meta': {'code': 200, 'requestId': '5d5f2a93342adf003884f753'},
 'response': {'venues': [{'id': '4f592f48e4b0c181e2e11cf7',
    'name': 'Starbucks',
    'location': {'lat': 34.02529780046195,
     'lng': -85.23876665506843,
     'labeledLatLngs': [{'label': 'display',
       'lat': 34.02529780046195,
       'lng': -85.23876665506843}],
     'distance': 16047,
     'cc': 'US',
     'state': 'Georgia',
     'country': 'United States',
     'formattedAddress': ['Georgia', 'United States']},
    'categories': [{'id': '4bf58dd8d48988d1e0931735',
      'name': 'Coffee Shop',
      'pluralName': 'Coffee Shops',
      'shortName': 'Coffee Shop',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1566517907',
    'hasPerk': False},
   {'id': '4bc44bccdce4eee1b113729d',
    'name': 'Quality Inn',
    'location': {'address': '925 N Main St',
     'lat': 34.03121400146373,
     'lng': -85.2473

In [185]:
cedartown_venues = cedartown_results['response']['venues']
cedartown_venues
# Tranform venues into a dataframe
cedartown_df = json_normalize(cedartown_venues)

filtered_columns = ['id','name', 'categories', 'location.lat', 'location.lng']
cedartown_df =cedartown_df.loc[:, filtered_columns]

# filter the category for each row
cedartown_df['categories'] = cedartown_df.apply(get_category_type, axis=1)

# clean columns
cedartown_df.columns = [col.split(".")[-1] for col in cedartown_df.columns]

cedartown_df

,id,name,categories,lat,lng
0,4f592f48e4b0c181e2e11cf7,Starbucks,Coffee Shop,34.025298,-85.238767
1,4bc44bccdce4eee1b113729d,Quality Inn,Hotel,34.031214,-85.247364
2,4bc39922dce4eee1deab719d,Holiday Inn Express & Suites Rome,Hotel,34.222323,-85.144741
3,4c427678ce54e21e13ce0c1a,Hampton Inn Jacksonville Anniston Area,Hotel,33.800980,-85.763873
4,4bc392fbdce4eee1caa9719d,Comfort Suites,Hotel,34.220993,-85.153311
5,4b71bfd4f964a520215a2de3,Java Jolt Coffee House,Coffee Shop,33.813750,-85.761112
6,4bc9fe19a276eee1643a6202,Americas Best Value Inn Rome,Hotel,34.177228,-85.205846
7,4fb9802de4b04d2c1d5512a2,Econo Lodge,Hotel,34.224800,-85.152670
8,4bc3b63ddce4eee1afb8719d,Best Western Executive Inn,Hotel,34.222950,-85.146528
9,4e98eaab0aaf0a73a1a082d2,Nuegene's Motor Inn,Bike Shop,33.816399,-85.746326


<b> Preparing the Cedartown dataset</b>

In [186]:
#List of trailheads  
#http://wikimapia.org/#lang=en&lat=33.893787&lon=-84.737549&z=12&m=w&show=/street/15589747/Silver-Comet-Bicycle-Trail
    
trailhead_list =[{'lat':33.8413889 ,'lng': -84.51666666666667 ,'trailhead':'Mavell Road Trailhead'},
            {'lat':33.9521417 ,'lng': -85.38857777777778 ,'trailhead':'Esom Hill Trailhead'},
            {'lat':33.9975 ,'lng':-85.05333333333333 ,'trailhead':'Rockmart Trailhead'},
            {'lat': 33.8444444,'lng':-84.53833333333333 ,'trailhead':'Fontaine Rd Trailhead'},
            {'lat':33.8569444 ,'lng':-84.65666666666667 ,'trailhead':'Carter Rd Trailhead'},
            {'lat':33.8522222 ,'lng':-84.55833333333334 ,'trailhead':'Concord Rd Trailhead'},
            {'lat':33.9066088 ,'lng':-84.8286064 ,'trailhead':'Dallas Trailhead'},
            {'lat':34.0132805  ,'lng':-85.2519375 ,'trailhead':'Cedartown Trailhead'}] 
trailhead_df =pd.DataFrame(trailhead_list)
trailhead_df

,lat,lng,trailhead
0,33.841389,-84.516667,Mavell Road Trailhead
1,33.952142,-85.388578,Esom Hill Trailhead
2,33.997500,-85.053333,Rockmart Trailhead
3,33.844444,-84.538333,Fontaine Rd Trailhead
4,33.856944,-84.656667,Carter Rd Trailhead
5,33.852222,-84.558333,Concord Rd Trailhead
6,33.906609,-84.828606,Dallas Trailhead
7,34.013281,-85.251937,Cedartown Trailhead


In [187]:
venues_map = folium.Map(location=[lat,long], zoom_start=11) # 

# add a yellow circle marker to represent selected trail head locations
for lat, lng, label in zip(trailhead_df.lat, trailhead_df.lng, trailhead_df.trailhead):
    folium.features.CircleMarker(
        [lat, lng],
      radius=5,
        color='yellow',
        popup=label,
        fill = True,
        fill_color='yellow',
         fill_opacity=0.8
    ).add_to(venues_map)



for lat, lng, label in zip(cedartown_df.lat, cedartown_df.lng, cedartown_df.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

for lat, lng, label in zip(smyrna_df_filtered.lat, smyrna_df_filtered.lng,  smyrna_df_filtered.categories):
     folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)
    
    
# display map
venues_map

In [188]:
cedartown_venue_id_list = cedartown_df['id'].tolist()
cedartown_venue_id_list

['4f592f48e4b0c181e2e11cf7',
 '4bc44bccdce4eee1b113729d',
 '4bc39922dce4eee1deab719d',
 '4c427678ce54e21e13ce0c1a',
 '4bc392fbdce4eee1caa9719d',
 '4b71bfd4f964a520215a2de3',
 '4bc9fe19a276eee1643a6202',
 '4fb9802de4b04d2c1d5512a2',
 '4bc3b63ddce4eee1afb8719d',
 '4e98eaab0aaf0a73a1a082d2',
 '4f579e3be4b04e88c1bf0b2c',
 '52e13e4211d24c87f2b670df',
 '5ab529b30d173f33415f58df',
 '5ba524333092be002c2fabd6',
 '4e9d811d8b81791d97bc2992',
 '509d642be4b0df8f7e4cec73',
 '4ee5e75699119449025cc1c8',
 '4f292266e4b0ea161de06c9c',
 '4bc33c71dce4eee19d81719d',
 '577db7ca498efe0b454cedb5',
 '4de28acfd22d2a4ecdc85959',
 '4fc18de1e4b09af80b2ee84a',
 '4e4ea1c018a8af30fd79727b',
 '4bc453dedce4eee18218729d',
 '4bc3e5d0dce4eee1d3cc719d',
 '513b4fb0e4b0c4aa125803a8',
 '4f03c0b56da1b97b15aa015c',
 '5259e2e411d2ebd2b7981a5c',
 '5604606b498eeb065ceccc52']

In [189]:
url_list = []
likes_count_list = []

for i in cedartown_venue_id_list:
    venue_url = 'https://api.foursquare.com/v2/venues/{}/likes?client_id={}&client_secret={}&v={}'.format(i, CLIENT_ID, CLIENT_SECRET, VERSION)
    url_list.append(venue_url)
for link in url_list:
    result = requests.get(link).json()
    likes = result['response']['likes']['count']
    likes_count_list.append(likes)
print(likes_count_list)

[0, 0, 15, 5, 5, 20, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 1, 0, 1, 0]


In [190]:
print(len(likes_count_list))
print(len(cedartown_venue_id_list))

29
29


In [191]:
# add in the likes count
cedartown_df['likes count'] = likes_count_list
cedartown_df.head()

,id,name,categories,lat,lng,likes count
0,4f592f48e4b0c181e2e11cf7,Starbucks,Coffee Shop,34.025298,-85.238767,0
1,4bc44bccdce4eee1b113729d,Quality Inn,Hotel,34.031214,-85.247364,0
2,4bc39922dce4eee1deab719d,Holiday Inn Express & Suites Rome,Hotel,34.222323,-85.144741,15
3,4c427678ce54e21e13ce0c1a,Hampton Inn Jacksonville Anniston Area,Hotel,33.800980,-85.763873,5
4,4bc392fbdce4eee1caa9719d,Comfort Suites,Hotel,34.220993,-85.153311,5


In [196]:
cedartown_df['grade']=cedartown_df.apply(cedartown_getGrade, axis=1)

In [197]:
cedartown_df.head()

,id,name,categories,lat,lng,likes count,grade
0,4f592f48e4b0c181e2e11cf7,Starbucks,Coffee Shop,34.025298,-85.238767,0,poor
1,4bc44bccdce4eee1b113729d,Quality Inn,Hotel,34.031214,-85.247364,0,poor
2,4bc39922dce4eee1deab719d,Holiday Inn Express & Suites Rome,Hotel,34.222323,-85.144741,15,good
3,4c427678ce54e21e13ce0c1a,Hampton Inn Jacksonville Anniston Area,Hotel,33.800980,-85.763873,5,average
4,4bc392fbdce4eee1caa9719d,Comfort Suites,Hotel,34.220993,-85.153311,5,average


In [198]:
# one hot encoding
to_onehot = pd.get_dummies(cedartown_df[['grade']], prefix="", prefix_sep="")

# add vaenue name column back to dataframe
to_onehot['Name'] = cedartown_df['name'] 

# move venue name column to the first column
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

to_onehot.head()

,Name,average,awesome,good,poor
0,Starbucks,0,0,0,1
1,Quality Inn,0,0,0,1
2,Holiday Inn Express & Suites Rome,0,0,1,0
3,Hampton Inn Jacksonville Anniston Area,1,0,0,0
4,Comfort Suites,1,0,0,0


In [199]:

cluster_df = to_onehot.drop('Name', axis=1)
k_clusters = 3

# run k-means clustering
kmeans = KMeans(n_clusters=k_clusters, random_state=0).fit(cluster_df)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 0, 2, 2, 0, 1, 1, 1, 1], dtype=int32)

In [200]:
cedartown_df['cluster'] = kmeans.labels_
cedartown_df.head(8)

,id,name,categories,lat,lng,likes count,grade,cluster
0,4f592f48e4b0c181e2e11cf7,Starbucks,Coffee Shop,34.025298,-85.238767,0,poor,1
1,4bc44bccdce4eee1b113729d,Quality Inn,Hotel,34.031214,-85.247364,0,poor,1
2,4bc39922dce4eee1deab719d,Holiday Inn Express & Suites Rome,Hotel,34.222323,-85.144741,15,good,0
3,4c427678ce54e21e13ce0c1a,Hampton Inn Jacksonville Anniston Area,Hotel,33.800980,-85.763873,5,average,2
4,4bc392fbdce4eee1caa9719d,Comfort Suites,Hotel,34.220993,-85.153311,5,average,2
5,4b71bfd4f964a520215a2de3,Java Jolt Coffee House,Coffee Shop,33.813750,-85.761112,20,awesome,0
6,4bc9fe19a276eee1643a6202,Americas Best Value Inn Rome,Hotel,34.177228,-85.205846,0,poor,1
7,4fb9802de4b04d2c1d5512a2,Econo Lodge,Hotel,34.224800,-85.152670,0,poor,1


In [201]:
nearby_venues_sort_by_likes = cedartown_df.sort_values('likes count', ascending=False)

In [209]:
map_clusters = folium.Map(location=[lat,long], zoom_start=12)

cedartown_venues = nearby_venues_sort_by_likes
smyrna_venues =Smyrna_nearby_venues_sort_by_likes
# set color scheme for the clusters
x = np.arange(k_clusters)
ys = [i+x+(i*x)**2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add a yellow circle marker to represent selected trail head locations
for lat, lng, label in zip(trailhead_df.lat, trailhead_df.lng, trailhead_df.trailhead):
    folium.features.CircleMarker(
        [lat, lng],
      radius=5,
        color='yellow',
        popup=label,
        fill = True,
        fill_color='yellow',
         fill_opacity=0.8
    ).add_to(map_clusters)

             
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(cedartown_venues['lat'], cedartown_venues['lng'], cedartown_venues['name'], nearby_venues_sort_by_likes['cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
        
for lat, lon, poi, cluster in zip(smyrna_venues['lat'], smyrna_venues['lng'], smyrna_venues['name'], Smyrna_nearby_venues_sort_by_likes['cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
        
map_clusters